Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# nnUNet MONAI Bundle

In this notebook, we will demonstrate how to create a MONAI Bundle supporting nnUNet experiment for training and inference. In this step-by step tutorial, we will describe how to create all the required python code and YAML configuration files needed to train and evaluate a nnUNet model using the MONAI Bundle format.

The tutorial assumes that the Spleen Dataset has been already downloaded and preprocessed as described in the [nnUNet MONAI Bundle Notebook](./06_nnunet_monai_bundle.ipynb).

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[pillow, tqdm]"
!python -c "import nnunetv2" || pip install -q nnunetv2

## Setup imports

In [ ]:
import torch
from monai.data import Dataset, DataLoader
from monai.handlers import (
    StatsHandler,
    from_engine,
    MeanDice,
    ValidationHandler,
    LrScheduleHandler,
    CheckpointSaver,
    CheckpointLoader,
    TensorBoardStatsHandler,
    MLFlowHandler,
)
from monai.engines import SupervisedTrainer, SupervisedEvaluator
from monai.transforms import Compose, Lambdad, Activationsd, AsDiscreted, Transposed, SaveImaged, LoadImaged

import re
import pathlib
import os
import yaml
import json
from monai.bundle import ConfigParser
import monai
from pathlib import Path
from odict import odict

from monai.bundle.nnunet import get_nnunet_trainer, get_nnunet_monai_predictor, convert_nnunet_to_monai_bundle

from monai.apps.nnunet import nnUNetV2Runner

from nnunetv2.utilities.dataset_name_id_conversion import maybe_convert_to_dataset_name
from nnunetv2.training.logging.nnunet_logger import nnUNetLogger
import shutil



In [ ]:
os.environ["MONAI_DATA_DIRECTORY"] = "MONAI/Data"

work_dir = os.path.join(os.environ["MONAI_DATA_DIRECTORY"], "nnUNet")

nnunet_raw = os.path.join(work_dir, "nnUNet_raw_data_base")
nnunet_preprocessed = os.path.join(".", work_dir, "nnUNet_preprocessed")
nnunet_results = os.path.join(".", work_dir, "nnUNet_trained_models")

if not os.path.exists(nnunet_raw):
    os.makedirs(nnunet_raw)

if not os.path.exists(nnunet_preprocessed):
    os.makedirs(nnunet_preprocessed)

if not os.path.exists(nnunet_results):
    os.makedirs(nnunet_results)

# claim environment variable
os.environ["nnUNet_raw"] = nnunet_raw
os.environ["nnUNet_preprocessed"] = nnunet_preprocessed
os.environ["nnUNet_results"] = nnunet_results
os.environ["OMP_NUM_THREADS"] = str(1)

## nnUNet Trainer

The core component for the nnUNet MONAI Bundle is the `get_nnunet_trainer` function. This function is responsible for creating the nnUNet trainer object from the native nnUNetv2 implementation. From the nnUNet trainer object, we can access the training components, such as the data loaders, model, learning rate scheduler, optimizer, and loss function, and perform training and inference tasks.

In [ ]:
nnunet_config = {
    "dataset_name_or_id": "009",
    "configuration": "3d_fullres",
    "trainer_class_name": "nnUNetTrainer_10epochs",
    "plans_identifier": "nnUNetPlans",
    "fold": 0,
}


nnunet_trainer = get_nnunet_trainer(**nnunet_config)

The function `get_nnunet_trainer` accepts the following parameters:

- `dataset_name_or_id`: The dataset name or ID to be used for training and evaluation.
- `fold`: The fold number for the cross-validation experiment.
- `configuration`: The training configuration for the nnUNet trainer, usually `3d_fullres`.
- `trainer_class_name`: The nnUNet trainer class name to be used for training, e.g. `nnUNetTrainer`.
- `plans_identifier`: The nnUNet plans identifier for the dataset, e.g. `nnUNetPlans`.

## Train and Val Data Loaders

In [ ]:
train_dataloader = nnunet_trainer.dataloader_train
train_data = [{"case_identifier": k} for k in nnunet_trainer.dataloader_train.generator._data.dataset.keys()]
train_dataset = Dataset(data=train_data)

In [ ]:
val_dataloader = nnunet_trainer.dataloader_val
val_data = [{"case_identifier": k} for k in nnunet_trainer.dataloader_val.generator._data.dataset.keys()]
val_dataset = Dataset(data=val_data)

## Network, Optimizer, and Loss Function

In [ ]:
device = nnunet_trainer.device

network = nnunet_trainer.network
optimizer = nnunet_trainer.optimizer
lr_scheduler = nnunet_trainer.lr_scheduler
loss = nnunet_trainer.loss

## Prepare Batch Function

The nnUnet `DataLoader` returns a dictionary with the `data` and `target` keys. Since the `SupervisedTrainer` used in the MONAI Bundle expects the data and target to be separate tensors, we need to create a custom prepare batch function to extract the data and target tensors from the dictionary.

In [ ]:
def prepare_nnunet_batch(batch, device, non_blocking):
    data = batch["data"].to(device, non_blocking=non_blocking)
    if isinstance(batch["target"], list):
        target = [i.to(device, non_blocking=non_blocking) for i in batch["target"]]
    else:
        target = batch["target"].to(device, non_blocking=non_blocking)
    return data, target

In [ ]:
image, label = prepare_nnunet_batch(next(iter(train_dataloader)), device="cpu", non_blocking=True)

## MONAI Supervised Trainer

The `SupervisedTrainer` class from MONAI is used to train the nnUNet model. For a minimal setup, we need to provide the model, optimizer, loss function, data loaders, number of epochs and the device to run the training.

In [ ]:
train_handlers = [StatsHandler(output_transform=from_engine(["loss"], first=True), tag_name="train_loss")]

In [ ]:
iterations = 100
epochs = 50

In [ ]:
trainer = SupervisedTrainer(
    amp=True,
    device=device,
    epoch_length=iterations,
    loss_function=loss,
    max_epochs=epochs,
    network=network,
    prepare_batch=prepare_nnunet_batch,
    optimizer=optimizer,
    train_data_loader=train_dataloader,
    train_handlers=train_handlers,
)

In [ ]:
trainer.run()

## Adding Validation and Validation Metrics

For a complete training setup, we need to add the validation data loader and the validation metrics to the `SupervisedTrainer`. Using the MONAI class `SupervisedEvaluator`, we can evaluate the model on the validation data loader and calculate the validation metrics (`Dice Score`).

In [ ]:
val_key_metric = MeanDice(output_transform=from_engine(["pred", "label"]), reduction="mean", include_background=False)

additional_metrics = {
    "Val_Dice_Per_Class": MeanDice(
        output_transform=from_engine(["pred", "label"]),
        reduction="mean_batch",
        include_background=False,
    )
}

Additionally, in order to compute the Mean Dice score over the batch, we need to apply a pos-processing transformtation to the nnUNet model output. Since `MeanDice` accepts `y` and `y_preds` as Batch-first tensors (BCHW[D]), we need to create a custom post-processing transform to convert the nnUNet model output to the required format.

In [ ]:
num_classes = 2

postprocessing = Compose(
    transforms=[
        ## Extract only high-res predictions from Deep Supervision
        Lambdad(keys=["pred", "label"], func=lambda x: x[0]),
        ## Apply Softmax to the predictions
        Activationsd(keys="pred", softmax=True),
        ## Binarize the predictions
        AsDiscreted(keys="pred", threshold=0.5),
        ## Convert the labels to one-hot
        AsDiscreted(keys="label", to_onehot=num_classes),
    ]
)

In [ ]:
val_handlers = [StatsHandler(iteration_log=False)]

In [ ]:
val_iterations = 100
val_interval = 1

In [ ]:
evaluator = SupervisedEvaluator(
    amp=True,
    device=device,
    epoch_length=val_iterations,
    network=network,
    key_val_metric={"Val_Dice": val_key_metric},
    prepare_batch=prepare_nnunet_batch,
    val_data_loader=val_dataloader,
    val_handlers=val_handlers,
    postprocessing=postprocessing,
    additional_metrics=additional_metrics,
)

And finally, we add the evaluator to the `SupervisedTrainer` to calculate the validation metrics during training.

In [ ]:
train_handlers.append(ValidationHandler(epoch_level=True, interval=val_interval, validator=evaluator))

We can also add the `MeanDice` metric to the `SupervisedTrainer` to calculate the mean dice score over the batch during training.

In [ ]:
train_key_metric = MeanDice(output_transform=from_engine(["pred", "label"]), reduction="mean", include_background=False)

additional_metrics = {
    "Train_Dice_Per_Class": MeanDice(
        output_transform=from_engine(["pred", "label"]),
        reduction="mean_batch",
        include_background=False,
    )
}

In [ ]:
trainer = SupervisedTrainer(
    amp=True,
    device=device,
    epoch_length=iterations,
    loss_function=loss,
    max_epochs=epochs,
    network=network,
    prepare_batch=prepare_nnunet_batch,
    optimizer=optimizer,
    train_data_loader=train_dataloader,
    train_handlers=train_handlers,
    key_train_metric={"Train_Dice": train_key_metric},
    postprocessing=postprocessing,
    additional_metrics=additional_metrics,
)

In [ ]:
trainer.run()

## Learning Rate Scheduler

One last component to add to the `SupervisedTrainer`, in order to replicate the training behaviour of the native nnUNet, is the learning rate scheduler.

In [ ]:
train_handlers.append(LrScheduleHandler(lr_scheduler=lr_scheduler, print_lr=True))

In [ ]:
trainer = SupervisedTrainer(
    amp=True,
    device=device,
    epoch_length=iterations,
    loss_function=loss,
    max_epochs=epochs,
    network=network,
    prepare_batch=prepare_nnunet_batch,
    optimizer=optimizer,
    train_data_loader=train_dataloader,
    train_handlers=train_handlers,
    key_train_metric={"Train_Dice": train_key_metric},
    postprocessing=postprocessing,
    additional_metrics=additional_metrics,
)

In [ ]:
trainer.run()

In [ ]:
train_handlers[-1].lr_scheduler.get_last_lr()

## Checkpointing

To save the model weights during training, we can use the `CheckpointSaver` callback from MONAI. This callback saves the model weights after each epoch.
We can later use the `CheckpointLoader` to load the model weights and perform inference or resume training.

In [ ]:
ckpt_dir = "nnUNetBundle/models"

val_handlers.append(
    CheckpointSaver(
        save_dir=ckpt_dir,
        save_dict={
            "network_weights": nnunet_trainer.network._orig_mod,
            "optimizer_state": nnunet_trainer.optimizer,
            "scheduler": nnunet_trainer.lr_scheduler,
        },
        # save_final= True,
        save_interval=1,
        save_key_metric=True,
        # final_filename= "model_final.pt",
        # key_metric_filename= "model.pt",
        n_saved=1,
    )
)

## Reload Checkpoint

When resuming the training from a checkpoint, we also want to restart the training from the same epoch. To do this, we need to load the checkpoint and update the `trainer.state.epoch` and `trainer.state.iteration` parameter in the `SupervisedTrainer`.

In [ ]:
def subfiles(folder, prefix=None, suffix=None, join=True, sort=True):
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    if prefix is not None:
        files = [f for f in files if f.startswith(prefix)]
    if suffix is not None:
        files = [f for f in files if f.endswith(suffix)]
    if sort:
        files.sort()
    if join:
        files = [os.path.join(folder, f) for f in files]
    return files


def get_checkpoint(epoch, ckpt_dir):
    if epoch == "latest":

        latest_checkpoints = subfiles(ckpt_dir, prefix="checkpoint_epoch", sort=True, join=False)
        epochs = []
        for latest_checkpoint in latest_checkpoints:
            epochs.append(int(latest_checkpoint[len("checkpoint_epoch=") : -len(".pt")]))

        epochs.sort()
        latest_epoch = epochs[-1]
        return latest_epoch
    else:
        return epoch


def reload_checkpoint(trainer, epoch, num_train_batches_per_epoch, ckpt_dir):

    epoch_to_load = get_checkpoint(epoch, ckpt_dir)
    trainer.state.epoch = epoch_to_load
    trainer.state.iteration = (epoch_to_load * num_train_batches_per_epoch) + 1

In [ ]:
reload_checkpoint_epoch = "latest"

train_handlers.append(
    CheckpointLoader(
        load_path=os.path.join(
            ckpt_dir, "checkpoint_epoch=" + str(get_checkpoint(reload_checkpoint_epoch, ckpt_dir)) + ".pt"
        ),
        load_dict={
            "network_weights": nnunet_trainer.network._orig_mod,
            "optimizer_state": nnunet_trainer.optimizer,
            "scheduler": nnunet_trainer.lr_scheduler,
        },
        map_location=device,
    )
)

## Initial nnUNet Checkpoint

In order to provide compatibility with the native nnUNet, we need to save the nnUNet-specific configuration, together the regular MONAI checkpoint. This is done only once, before the training starts. At the end of the training, we will have a MONAI checkpoint and a nnUNet checkpoint. To be able to convert the MONAI checkpoint to a nnUNet checkpoint at any time, we can then combine the two checkpoints.

In [ ]:
checkpoint = {
    "inference_allowed_mirroring_axes": nnunet_trainer.inference_allowed_mirroring_axes,
    "init_args": nnunet_trainer.my_init_kwargs,
    "trainer_name": nnunet_trainer.__class__.__name__,
}
checkpoint_filename = os.path.join(ckpt_dir, "nnunet_checkpoint.pth")

torch.save(checkpoint, checkpoint_filename)

## MLFlow and Tensorboard Monitoring

To monitor the training process, we can use MLFlow and Tensorboard. We can log the training metrics, hyperparameters, and model weights to MLFlow, and visualize the training metrics using Tensorboard.

In [ ]:
log_dir = "nnUNetBundle/logs"

train_handlers.append(
    TensorBoardStatsHandler(log_dir=log_dir, output_transform=from_engine(["loss"], first=True), tag_name="train_loss")
)

val_handlers.append(TensorBoardStatsHandler(log_dir=log_dir, iteration_log=False))

In [ ]:
def mlflow_transform(state_output):
    return state_output[0]["loss"]


class MLFlownnUNetHandler(MLFlowHandler):
    def __init__(self, label_dict, **kwargs):
        super(MLFlownnUNetHandler, self).__init__(**kwargs)
        self.label_dict = label_dict

    def _default_epoch_log(self, engine) -> None:
        """
        Execute epoch level log operation.
        Default to track the values from Ignite `engine.state.metrics` dict and
        track the values of specified attributes of `engine.state`.

        Args:
            engine: Ignite Engine, it can be a trainer, validator or evaluator.

        """
        log_dict = engine.state.metrics
        if not log_dict:
            return

        current_epoch = self.global_epoch_transform(engine.state.epoch)

        new_log_dict = {}

        for metric in log_dict:
            if type(log_dict[metric]) == torch.Tensor:
                for i, val in enumerate(log_dict[metric]):
                    new_log_dict[metric + "_{}".format(list(self.label_dict.keys())[i + 1])] = val
            else:
                new_log_dict[metric] = log_dict[metric]
        self._log_metrics(new_log_dict, step=current_epoch)

        if self.state_attributes is not None:
            attrs = {attr: getattr(engine.state, attr, None) for attr in self.state_attributes}
            self._log_metrics(attrs, step=current_epoch)

In [ ]:
def create_mlflow_experiment_params(params_file, custom_params=None):
    params_dict = {}
    config_values = monai.config.deviceconfig.get_config_values()
    for k in config_values:
        params_dict[re.sub("[()]", " ", str(k))] = config_values[k]

    optional_config_values = monai.config.deviceconfig.get_optional_config_values()
    for k in optional_config_values:
        params_dict[re.sub("[()]", " ", str(k))] = optional_config_values[k]

    gpu_info = monai.config.deviceconfig.get_gpu_info()
    for k in gpu_info:
        params_dict[re.sub("[()]", " ", str(k))] = str(gpu_info[k])

    yaml_config_files = [params_file]
    # %%
    monai_config = {}
    for config_file in yaml_config_files:
        with open(config_file, "r") as file:
            monai_config.update(yaml.safe_load(file))

    monai_config["bundle_root"] = str(Path(Path(params_file).parent).parent)

    parser = ConfigParser(monai_config, globals={"os": "os", "pathlib": "pathlib", "json": "json", "ignite": "ignite"})

    parser.parse(True)

    for k in monai_config:
        params_dict[k] = parser.get_parsed_content(k, instantiate=True)

    if custom_params is not None:
        for k in custom_params:
            params_dict[k] = custom_params[k]
    return params_dict

In [ ]:
%%writefile nnUNetBundle/mlflow_params.yaml

dataset_name_or_id: "009"
nnunet_trainer_class_name: "nnUNetTrainer"
nnunet_plans_identifier: "nnUNetPlans"

num_classes: 2
label_dict:
    0: "background"
    1: "spleen"
    
tracking_uri: "http://localhost:5000"
mlflow_experiment_name: "nnUNet_Bundle_Spleen"
mlflow_run_name: "nnUNet_Bundle_Spleen"




In [ ]:
mlflow_experiment_name = "nnUNet_Bundle_Spleen"
mlflow_run_name = "nnUNet_Bundle_Spleen"
label_dict = {0: "background", 1: "Spleen"}
tracking_uri = "http://localhost:5000"

params_file = "nnUNetBundle/mlflow_params.yaml"


train_handlers.append(
    MLFlownnUNetHandler(
        dataset_dict={"train": train_dataset},
        dataset_keys="case_identifier",
        experiment_param=create_mlflow_experiment_params(params_file),
        experiment_name=mlflow_experiment_name,
        label_dict=label_dict,
        output_transform=mlflow_transform,
        run_name=mlflow_run_name,
        state_attributes=["best_metric", "best_metric_epoch"],
        tag_name="Train_Loss",
        tracking_uri=tracking_uri,
    )
)

val_handlers.append(
    MLFlownnUNetHandler(
        experiment_name=mlflow_experiment_name,
        iteration_log=False,
        label_dict=label_dict,
        output_transform=mlflow_transform,
        run_name=mlflow_run_name,
        state_attributes=["best_metric", "best_metric_epoch"],
        tracking_uri=tracking_uri,
    )
)

To start the MLFlow server, we can run the following command in the terminal:

```bash
cd nnUNetBundle/MLFlow && mlflow server
```
To run Tensorboard, we can use the following command:

```bash
tensorboard --logdir Bundle/logs
```

In [ ]:
trainer = SupervisedTrainer(
    amp=True,
    device=device,
    epoch_length=iterations,
    loss_function=loss,
    max_epochs=epochs,
    network=network,
    prepare_batch=prepare_nnunet_batch,
    optimizer=optimizer,
    train_data_loader=train_dataloader,
    train_handlers=train_handlers,
    key_train_metric={"Train_Dice": train_key_metric},
    postprocessing=postprocessing,
    additional_metrics=additional_metrics,
)

In [ ]:
trainer.run()

## Create MONAI Bundle

In [ ]:
%%bash 

python -m monai.bundle init_bundle nnUNetBundle

mkdir -p nnUNetBundle/nnUNet
mkdir -p nnUNetBundle/src
mkdir -p nnUNetBundle/nnUNet/evaluator
which tree && tree nnUNetBundle || true

In [ ]:
%%writefile nnUNetBundle/configs/logging.conf
[loggers]
keys=root

[handlers]
keys=consoleHandler

[formatters]
keys=fullFormatter

[logger_root]
level=INFO
handlers=consoleHandler

[handler_consoleHandler]
class=StreamHandler
level=INFO
formatter=fullFormatter
args=(sys.stdout,)

[formatter_fullFormatter]
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s


In [ ]:
%%writefile nnUNetBundle/configs/metadata.json

#TODO: Add MetaData, following the instructions in https://docs.monai.io/en/stable/mb_specification.html

In [ ]:
%%writefile nnUNetBundle/nnUNet/global.yaml

iterations: $@nnunet_trainer.num_iterations_per_epoch
device: $@nnunet_trainer.device
epochs: $@nnunet_trainer.num_epochs

bundle_root: .
ckpt_dir: "$@bundle_root + '/models'"

In [ ]:
%%writefile nnUNetBundle/nnUNet/params.yaml


dataset_name_or_id: "009"
nnunet_trainer_class_name: "nnUNetTrainer"
nnunet_plans_identifier: "nnUNetPlans"
nnunet_configuration: "3d_fullres"

num_classes: 2
label_dict:
    0: "background"
    1: "spleen"
    
tracking_uri: "http://localhost:5000"
mlflow_experiment_name: "nnUNet_Bundle_Spleen"
mlflow_run_name: "nnUNet_Bundle_Spleen"
log_dir: "$@bundle_root + '/logs'"

In [ ]:
%%writefile nnUNetBundle/nnUNet/imports.yaml

imports:
- $import glob
- $import os
- $import ignite
- $import torch
- $import shutil
- $import json
- $import src
- $import nnunetv2
- $import src.mlflow
- $import src.trainer
- $from pathlib import Path

In [ ]:
%%writefile nnUNetBundle/nnUNet/run.yaml

run:
- "$torch.save(@checkpoint,@checkpoint_filename)"
- "$shutil.copy(Path(@nnunet_model_folder).joinpath('dataset.json'), @bundle_root+'/models/dataset.json')"
- "$shutil.copy(Path(@nnunet_model_folder).joinpath('plans.json'), @bundle_root+'/models/plans.json')"
- "$@train#pbar.attach(@train#trainer,output_transform=lambda x: {'loss': x[0]['loss']})"
- "$@validate#pbar.attach(@validate#evaluator,metric_names=['Val_Dice'])"
- $@train#trainer.run()

initialize:
- $monai.utils.set_determinism(seed=123)

In [ ]:
%%writefile nnUNetBundle/nnUNet/nnunet_trainer.yaml

nnunet_trainer:
  _target_ : get_nnunet_trainer
  dataset_name_or_id: "@dataset_name_or_id"
  configuration: "@nnunet_configuration"
  fold: 0
  trainer_class_name: "@nnunet_trainer_class_name"
  plans_identifier: "@nnunet_plans_identifier"

loss: $@nnunet_trainer.loss
lr_scheduler: $@nnunet_trainer.lr_scheduler

network: $@nnunet_trainer.network

optimizer: $@nnunet_trainer.optimizer

checkpoint:
  init_args: '$@nnunet_trainer.my_init_kwargs'
  trainer_name: '$@nnunet_trainer.__class__.__name__'
  inference_allowed_mirroring_axes: '$@nnunet_trainer.inference_allowed_mirroring_axes'

checkpoint_filename: "$@bundle_root+'/models/nnunet_checkpoint.pth'"

dataset_name: "$nnunetv2.utilities.dataset_name_id_conversion.maybe_convert_to_dataset_name(@dataset_name_or_id)"
nnunet_model_folder: "$os.path.join(os.environ['nnUNet_results'], @dataset_name, @nnunet_trainer_class_name+'__'+@nnunet_plans_identifier+'__'+@nnunet_configuration)"

In [ ]:
%%writefile nnUNetBundle/nnUNet/train_metrics.yaml

train_key_metric:
  Train_Dice:
    _target_: "MeanDice"
    include_background: False
    output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
    reduction: "mean"

train_additional_metrics:
  Train_Dice_per_class:
    _target_: "MeanDice"
    include_background: False
    output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
    reduction: "mean_batch"

In [ ]:
%%writefile nnUNetBundle/nnUNet/train_postprocessing.yaml

train_postprocessing:
  _target_: "Compose"
  transforms:
  - _target_: Lambdad
    keys:
      - "pred"
      - "label"
    func: "$lambda x: x[0]"
  - _target_: Activationsd
    keys:
      - "pred"
    softmax: True
  - _target_: AsDiscreted
    keys:
      - "pred"
    threshold: 0.5
  - _target_: AsDiscreted
    keys:
      - "label"
    to_onehot: "@num_classes"
    
train_postprocessing_region_based:
  _target_: "Compose"
  transforms:
  - _target_: Lambdad
    keys:
      - "pred"
      - "label"
    func: "$lambda x: x[0]"
  - _target_: Activationsd
    keys:
      - "pred"
    sigmoid: True
  - _target_: AsDiscreted
    keys:
      - "pred"
    threshold: 0.5

In [ ]:
%%writefile nnUNetBundle/nnUNet/train.yaml

train:
  pbar:
    _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
  dataloader: $@nnunet_trainer.dataloader_train
  train_data: "$[{'case_identifier':k} for k in @nnunet_trainer.dataloader_train.generator._data.dataset.keys()]"
  train_dataset:
    _target_: Dataset
    data: "@train#train_data"
  inferer:
    _target_: SimpleInferer
  trainer:
    _target_: SupervisedTrainer
    amp: true
    device: '@device'
    additional_metrics: "@train_additional_metrics"
    epoch_length: "@iterations"
    inferer: '@train#inferer'
    key_train_metric: '@train_key_metric'
    loss_function: '@loss'
    max_epochs: '@epochs'
    network: '@network'
    prepare_batch: "$src.trainer.prepare_nnunet_batch"
    optimizer: '@optimizer'
    postprocessing: '@train_postprocessing'
    train_data_loader: '@train#dataloader'
    train_handlers: '@train_handlers#handlers'

In [ ]:
%%writefile nnUNetBundle/nnUNet/train_handlers.yaml

train_handlers:
  handlers:
  - _target_: "$src.mlflow.MLFlownnUNetHandler"
    label_dict: "@label_dict"
    tracking_uri: "@tracking_uri"
    experiment_name: "@mlflow_experiment_name"
    run_name: "@mlflow_run_name"
    output_transform: "$src.mlflow.mlflow_transform"
    dataset_dict:
        train: "@train#train_dataset"
    dataset_keys: 'case_identifier'
    state_attributes:
    - "iteration"
    - "epoch"
    tag_name: 'Train_Loss'
    experiment_param: "$src.mlflow.create_mlflow_experiment_params( @bundle_root + '/nnUNet/params.yaml')"
    #artifacts=None
    optimizer_param_names: 'lr'
    #close_on_complete: False
  - _target_: LrScheduleHandler
    lr_scheduler: '@lr_scheduler'
    print_lr: true
  - _target_: ValidationHandler
    epoch_level: true
    interval: '@val_interval'
    validator: '@validate#evaluator'
  #- _target_: StatsHandler
  #  output_transform: $monai.handlers.from_engine(['loss'], first=True)
  #  tag_name: train_loss
  - _target_: TensorBoardStatsHandler
    log_dir: '@log_dir'
    output_transform: $monai.handlers.from_engine(['loss'], first=True)
    tag_name: train_loss

In [ ]:
%%writefile nnUNetBundle/configs/train_resume.yaml

run:
- '$src.trainer.reload_checkpoint(@train#trainer,@reload_checkpoint_epoch,@nnunet_trainer.num_iterations_per_epoch,@bundle_root+"/models")'
- "$@train#pbar.attach(@train#trainer,output_transform=lambda x: {'loss': x[0]['loss']})"
- "$@validate#pbar.attach(@validate#evaluator,metric_names=['Val_Dice'])"
- $@train#trainer.run()

train_handlers:
  handlers:
  - _target_: "$src.mlflow.MLFlownnUNetHandler"
    label_dict: "@label_dict"
    tracking_uri: "@tracking_uri"
    experiment_name: "@mlflow_experiment_name"
    run_name: "@mlflow_run_name"
    output_transform: "$src.mlflow.mlflow_transform"
    dataset_dict:
        train: "@train#train_dataset"
    dataset_keys: 'case_identifier'
    state_attributes:
    - "iteration"
    - "epoch"
    tag_name: 'Train_Loss'
    experiment_param: "$src.mlflow.create_mlflow_experiment_params( @bundle_root + '/nnUNet/params.yaml')"
    #artifacts=None
    optimizer_param_names: 'lr'
    #close_on_complete: False
  - _target_: LrScheduleHandler
    lr_scheduler: '@lr_scheduler'
    print_lr: true
  - _target_: ValidationHandler
    epoch_level: true
    interval: '@val_interval'
    validator: '@validate#evaluator'
  #- _target_: StatsHandler
  #  output_transform: $monai.handlers.from_engine(['loss'], first=True)
  #  tag_name: train_loss
  - _target_: TensorBoardStatsHandler
    log_dir: '@log_dir'
    output_transform: $monai.handlers.from_engine(['loss'], first=True)
    tag_name: train_loss
  - _target_: CheckpointLoader
    load_dict:
      network_weights: '$@nnunet_trainer.network._orig_mod'
      optimizer_state: '$@nnunet_trainer.optimizer'
      scheduler: '$@nnunet_trainer.lr_scheduler'
    load_path: '$@bundle_root + "/models/checkpoint_epoch="+str(src.trainer.get_checkpoint(@reload_checkpoint_epoch, @bundle_root+"/models"))+".pt"'
    map_location: '@device'

In [ ]:
%%writefile nnUNetBundle/nnUNet/val_metrics.yaml

val_key_metric:
  Val_Dice:
    _target_: "MeanDice"
    output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
    reduction: "mean"
    include_background: False
    
val_additional_metrics:
  Val_Dice_per_class:
    _target_: "MeanDice"
    output_transform: "$monai.handlers.from_engine(['pred', 'label'])"
    reduction: "mean_batch"
    include_background: False

In [ ]:
%%writefile nnUNetBundle/nnUNet/val_handlers.yaml

val_handlers:
  handlers:
  - _target_: StatsHandler
    iteration_log: false
  - _target_: TensorBoardStatsHandler
    iteration_log: false
    log_dir: '@log_dir'
  - _target_: "$src.mlflow.MLFlownnUNetHandler"
    label_dict: "@label_dict"
    tracking_uri: "@tracking_uri"
    experiment_name: "@mlflow_experiment_name"
    run_name: "@mlflow_run_name"
    output_transform: "$src.mlflow.mlflow_transform"
    iteration_log: False
    state_attributes:
    - "best_metric"
    - "best_metric_epoch"
  - _target_: "CheckpointSaver"
    save_dir: "$str(@bundle_root)+'/models'"
    save_interval: 1
    n_saved: 1
    save_key_metric: true
    save_dict:
      network_weights: '$@nnunet_trainer.network._orig_mod'
      optimizer_state: '$@nnunet_trainer.optimizer'
      scheduler: '$@nnunet_trainer.lr_scheduler'

In [ ]:
%%writefile nnUNetBundle/nnUNet/validate.yaml

val_interval: 1
validate:
  pbar:
    _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
  dataloader: $@nnunet_trainer.dataloader_val
  evaluator:
    _target_: SupervisedEvaluator
    additional_metrics: '@val_additional_metrics'
    amp: true
    epoch_length: $@nnunet_trainer.num_val_iterations_per_epoch
    device: '@device'
    inferer: '@validate#inferer'
    key_val_metric: '@val_key_metric'
    network: '@network'
    postprocessing: '@train_postprocessing'
    val_data_loader: '@validate#dataloader'
    val_handlers: '@val_handlers#handlers'
    prepare_batch: "$src.trainer.prepare_nnunet_batch"
  inferer:
    _target_: SimpleInferer


In [ ]:
%%writefile nnUNetBundle/nnUNet/evaluator/evaluator.yaml

#Remove CheckpointSaver from val_handlers

run:
- "$@validate#pbar.attach(@validate#evaluator,metric_names=['Val_Dice'])"
- $@validate#evaluator.run()

initialize:
- "$setattr(torch.backends.cudnn, 'benchmark', True)"

## Adding Python Utility Scripts

We finally add the MLFlow and Training utility scripts to the MONAI Bundle.

In [ ]:
%%writefile nnUNetBundle/src/__init__.py



In [ ]:
%%writefile nnUNetBundle/src/mlflow.py

import re
from monai.handlers import  MLFlowHandler
import yaml
from monai.bundle import ConfigParser
from pathlib import Path
import monai
import torch

def mlflow_transform(state_output):
    """
    Extracts the 'loss' value from the first element of the state_output list.

    Parameters
    ----------
    state_output : list of dict
        A list where each element is a dictionary containing various metrics, including 'loss'.

    Returns
    -------
    float
        The 'loss' value from the first element of the state_output list.
    """
    return state_output[0]['loss']

class MLFlownnUNetHandler(MLFlowHandler):
    """
    A handler for logging nnUNet metrics to MLFlow.
    Parameters
    ----------
    label_dict : dict
        A dictionary mapping label indices to label names.
    **kwargs : dict
        Additional keyword arguments passed to the parent class.
    """
    def __init__(self, label_dict, **kwargs):
        super(MLFlownnUNetHandler, self).__init__(**kwargs)
        self.label_dict = label_dict
        
    def _default_epoch_log(self, engine) -> None:
        """
        Logs the metrics and state attributes at the end of each epoch.

        Parameters
        ----------
        engine : Engine
            The engine object that contains the state and metrics to be logged.

        Returns
        -------
        None
        """
        log_dict = engine.state.metrics
        if not log_dict:
            return

        current_epoch = self.global_epoch_transform(engine.state.epoch)

        new_log_dict = {}

        for metric in log_dict:
            if type(log_dict[metric]) == torch.Tensor:
                for i,val in enumerate(log_dict[metric]):
                    new_log_dict[metric+"_{}".format(list(self.label_dict.keys())[i+1])] = val
            else:
                new_log_dict[metric] = log_dict[metric]
        self._log_metrics(new_log_dict, step=current_epoch)

        if self.state_attributes is not None:
            attrs = {attr: getattr(engine.state, attr, None) for attr in self.state_attributes}
            self._log_metrics(attrs, step=current_epoch)

def create_mlflow_experiment_params(params_file, custom_params=None):
    """
    Create a dictionary of parameters for an MLflow experiment.

    This function reads configuration values from MONAI, GPU information, and a YAML configuration file,
    and combines them into a single dictionary. Optionally, custom parameters can also be added to the dictionary.

    Parameters
    ----------
    params_file : str
        Path to the YAML configuration file.
    custom_params : dict, optional
        A dictionary of custom parameters to be added to the final parameters dictionary (default is None).

    Returns
    -------
    dict
        A dictionary containing all the combined parameters.
    """
    params_dict = {}
    config_values = monai.config.deviceconfig.get_config_values()
    for k in config_values:
        params_dict[re.sub("[()]"," ",str(k))] = config_values[k]

    optional_config_values = monai.config.deviceconfig.get_optional_config_values()
    for k in optional_config_values:
        params_dict[re.sub("[()]"," ",str(k))] = optional_config_values[k]

    gpu_info = monai.config.deviceconfig.get_gpu_info()
    for k in gpu_info:
        params_dict[re.sub("[()]"," ",str(k))] = str(gpu_info[k])

    yaml_config_files = [params_file]
    # %%
    monai_config = {}
    for config_file in yaml_config_files:
        with open(config_file, 'r') as file:
            monai_config.update(yaml.safe_load(file))

    monai_config["bundle_root"] = str(Path(Path(params_file).parent).parent)

    parser = ConfigParser(monai_config, globals={"os": "os",
                                                 "pathlib": "pathlib",
                                                 "json": "json",
                                                 "ignite": "ignite"
                                                 })

    parser.parse(True)

    for k in monai_config:
        params_dict[k] = parser.get_parsed_content(k,instantiate=True)

    if custom_params is not None:
        for k in custom_params:
            params_dict[k] = custom_params[k]
    return params_dict


In [ ]:
%%writefile nnUNetBundle/src/trainer.py

import os

def subfiles(directory, prefix=None, suffix=None, join=True, sort=True):
    """
    List files in a directory with optional filtering by prefix and/or suffix.
    
    Parameters
    ----------
    directory : str
        The path to the directory to list files from.
    prefix : str, optional
        If specified, only files starting with this prefix will be included.
    suffix : str, optional
        If specified, only files ending with this suffix will be included.
    join : bool, optional
        If True, the directory path will be joined with the filenames. Default is True.
    sort : bool, optional
        If True, the list of files will be sorted. Default is True.
    
    Returns
    -------
    list of str
        A list of filenames (with full paths if `join` is True) that match the specified criteria.
    """

    
    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if prefix is not None:
        files = [f for f in files if f.startswith(prefix)]
    if suffix is not None:
        files = [f for f in files if f.endswith(suffix)]
    if join:
        files = [os.path.join(directory, f) for f in files]
    if sort:
        files.sort()
    return files

def prepare_nnunet_batch(batch, device, non_blocking):
    """
    Prepares a batch of data and targets for nnU-Net training by transferring them to the specified device.

    Parameters
    ----------
    batch : dict
        A dictionary containing the data and target tensors. The key "data" corresponds to the input data tensor,
        and the key "target" corresponds to the target tensor or a list of target tensors.
    device : torch.device
        The device to which the data and target tensors should be transferred (e.g., 'cuda' or 'cpu').
    non_blocking : bool
        If True, allows non-blocking data transfer to the device.

    Returns
    -------
    tuple
        A tuple containing the data tensor and the target tensor(s) after being transferred to the specified device.
    """
    data = batch["data"].to(device, non_blocking=non_blocking)
    if isinstance(batch["target"], list):
        target = [i.to(device, non_blocking=non_blocking) for i in batch["target"]]
    else:
        target = batch["target"].to(device, non_blocking=non_blocking)
    return data, target

def get_checkpoint(epoch, ckpt_dir):
    """
    Retrieves the checkpoint for a given epoch from the checkpoint directory.

    Parameters
    ----------
    epoch : int or str
        The epoch number to retrieve. If 'latest', the function will return the latest checkpoint.
    ckpt_dir : str
        The directory where checkpoints are stored.

    Returns
    -------
    int
        The epoch number of the checkpoint to be retrieved. If 'latest', returns the latest epoch number.
    """
    if epoch == "latest":

        latest_checkpoints = subfiles(ckpt_dir, prefix="checkpoint_epoch", sort=True,
                                      join=False)
        epochs = []
        for latest_checkpoint in latest_checkpoints:
            epochs.append(int(latest_checkpoint[len("checkpoint_epoch="):-len(".pt")]))

        epochs.sort()
        latest_epoch = epochs[-1]
        return latest_epoch
    else:
        return epoch

def reload_checkpoint(trainer, epoch, num_train_batches_per_epoch, ckpt_dir):
    """
    Reloads the checkpoint for a given epoch and updates the trainer's state.

    Parameters
    ----------
    trainer : object
        The trainer object whose state needs to be updated.
    epoch : int
        The epoch number to load the checkpoint from.
    num_train_batches_per_epoch : int
        The number of training batches per epoch.
    ckpt_dir : str
        The directory where the checkpoints are stored.

    Returns
    -------
    None
    """

    epoch_to_load = get_checkpoint(epoch, ckpt_dir)
    trainer.state.epoch = epoch_to_load
    trainer.state.iteration = (epoch_to_load* num_train_batches_per_epoch) +1


In [ ]:
def create_config(config_folder, output_file):
    config_files = [f.path for f in os.scandir(config_folder) if f.path.endswith(".yaml")]
    config = {}
    for config_file in config_files:
        with open(config_file, "r") as file:
            config.update(yaml.safe_load(file))

    if output_file.endswith(".yaml"):
        with open(output_file, "w") as file:
            yaml.dump(config, file)
    if output_file.endswith(".json"):
        with open(output_file, "w") as file:
            json.dump(config, file)

    return config

In [ ]:
config = create_config("nnUNetBundle/nnUNet", "nnUNetBundle/configs/train.yaml")

In [ ]:
%%bash

export BUNDLE_ROOT=nnUNetBundle
export PYTHONPATH=$PYTHONPATH:$BUNDLE_ROOT

python -m monai.bundle run \
    --bundle_root $BUNDLE_ROOT \
    --config_file $BUNDLE_ROOT/configs/train.yaml

# Option to resume training
#--config_file "['$BUNDLE_ROOT/configs/train.yaml','$BUNDLE_ROOT/configs/train_resume.yaml']"

# Log to Local MLFlow
#--tracking_uri mlruns


## Inference

After training the nnUNet model, we can then perform inference on new data. We use a `ModelnnUNetWrapper` as a wrapper around the nnUNet model to perform inference from the MONAI Bundle. In this way, the nnUNet preprocessing, inference and postprocessing steps are handled by the `ModelnnUNetWrapper`, with the Bundle blocks only needing to handle the input data loading and sending to the nnUnet block and the nnUNet prediction postprocessing.

The `ModelnnUNetWrapper` receives as input the data dictionary loaded by the DataLoader, and returns the model predictions as a MetaTensor.

To get the `ModelnnUNetWrapper` object, we can use the `get_nnunet_monai_predictor` function, which receives the following parameters:

- `model_folder`: The path to the nnUNet model folder.
- `model_name`: [Optional] The name of the model to be loaded. If not provided, the function will load the checkpoint named `model.pt`.

In [ ]:
# To Select the lastest checkpoint

from nnUNetBundle.src.trainer import get_checkpoint

ckpt_epoch = get_checkpoint("latest", "nnUNetBundle/models")

In [ ]:
nnunet_config = {
    "model_folder": "nnUNetBundle/models",
}

monai_predictor = get_nnunet_monai_predictor(**nnunet_config, model_name=f"checkpoint_epoch={ckpt_epoch}.pt")

## Test Data Preparation

The Bundle accepts the test dataset in the following format:

```bash
Dataset
├── Case1
│   └── Case1.nii.gz
├── Case2
│   └── Case2.nii.gz
└── Case3
    └── Case3.nii.gz
```

In [ ]:
%%bash

mkdir -p nnUNetBundle/test_input/spleen_1
mkdir -p nnUNetBundle/test_output

cp MONAI/Data/Task09_Spleen/imagesTs/spleen_1.nii.gz nnUNetBundle/test_input/spleen_1

In [ ]:
%%bash

tree nnUNetBundle/test_input

### Data Loading

In [ ]:
def get_subfolder_dataset(data_dir, modality_conf):
    data_list = []
    for f in os.scandir(data_dir):

        if f.is_dir():
            subject_dict = {
                key: str(pathlib.Path(f.path).joinpath(f.name + modality_conf[key]["suffix"])) for key in modality_conf
            }
            data_list.append(subject_dict)
    return data_list

In [ ]:
modalities = {
    "image": {"suffix": ".nii.gz"},
}

data = get_subfolder_dataset("nnUNetBundle/test_input", modalities)

In [ ]:
preprocessing = LoadImaged(keys=["image"], ensure_channel_first=True, image_only=False)


test_dataset = Dataset(data, transform=preprocessing)

test_loader = DataLoader(test_dataset, batch_size=1)

### Test ModelnnUNetWrapper

To test the `ModelnnUNetWrapper`, we can provide a test case to the `ModelnnUNetWrapper` and extract the model predictions returned by the wrapper.

In [ ]:
batch = next(iter(test_loader))

pred = monai_predictor(batch["image"])

### Postprocessing and Save Predictions

After obtaining the model predictions, we can apply postprocessing transformations to the predictions and save the results to disk.

The `Transposed` transform is required to unify the axis order convention between MONAI and nnUNet. The nnUNet model uses the `zyx` axis order, while MONAI uses the `xyz` axis order.

In [ ]:
postprocessing = Compose(
    [
        # Decollated(keys=None, detach=True),
        Transposed(keys="pred", indices=[0, 3, 2, 1]),
        SaveImaged(
            keys="pred",
            output_dir="nnUNetBundle/test_output",
            output_postfix="prediction",
            meta_keys="image_meta_dict",
        ),
    ]
)

In [ ]:
postprocessing({"pred": pred})

## Evaluator

Combining everything together, we can create an `Evaluator` that encapsulates the data loading, model inference, postprocessing, and evaluation steps. The `Evaluator` can be used to evaluate the model on the test dataset .

In [ ]:
validator = SupervisedEvaluator(
    val_data_loader=test_loader, device="cuda:0", network=monai_predictor, postprocessing=postprocessing
)

In [ ]:
validator.run()

In [ ]:
%%writefile nnUNetBundle/configs/inference.yaml

imports: 
  - $import json
  - $from pathlib import Path
  - $import os
  - $import monai.bundle.nnunet
  - $from ignite.contrib.handlers.tqdm_logger import ProgressBar
  - $import shutil
  - $import src
  - $import src.dataset


output_dir: "."
bundle_root: "."
data_list_file : "."
data_dir: "."

prediction_suffix: "prediction"


modality_conf:
  image:
    suffix: ".nii.gz"

test_data_list: "$src.dataset.get_subfolder_dataset(@data_dir,@modality_conf)"
#test_data_list: "$monai.data.load_decathlon_datalist(@data_list_file, is_segmentation=True, data_list_key='testing', base_dir=@data_dir)"
image_modality_keys: "$list(@modality_conf.keys())"
image_key: "image"
image_suffix: "@image_key"

preprocessing:
  _target_: Compose
  transforms:
  - _target_: LoadImaged
    keys: "image"
    ensure_channel_first: True
    image_only: False

test_dataset:
  _target_: Dataset
  data: "$@test_data_list"
  transform: "@preprocessing"

test_loader:
  _target_: DataLoader
  dataset: "@test_dataset"
  batch_size: 1


device: "$torch.device('cuda')"

nnunet_config:
  model_folder: "$@bundle_root + '/models'"

network_def: "$monai.bundle.nnunet.get_nnunet_monai_predictor(**@nnunet_config)"

postprocessing:
  _target_: "Compose"
  transforms:
    - _target_: Transposed
      keys: "pred"
      indices:
      - 0
      - 3
      - 2
      - 1
    - _target_: SaveImaged
      keys: "pred"
      resample: False
      output_postfix: "@prediction_suffix"
      output_dir: "@output_dir"
      meta_keys: "image_meta_dict"


testing:
  dataloader: "$@test_loader"
  pbar:
    _target_: "ignite.contrib.handlers.tqdm_logger.ProgressBar"
  test_inferer: "$@inferer"

inferer: 
  _target_: "SimpleInferer"

validator:
  _target_: "SupervisedEvaluator"
  postprocessing: "$@postprocessing"
  device: "$@device"
  inferer: "$@testing#test_inferer"
  val_data_loader: "$@testing#dataloader"
  network: "@network_def"
  val_handlers:
  - _target_: "CheckpointLoader"
    load_path: "$@bundle_root+'/models/model.pt'"
    load_dict:
      network_weights: '$@network_def.network_weights'
run:
  - "$@testing#pbar.attach(@validator)"
  - "$@validator.run()"

In [ ]:
%%writefile nnUNetBundle/src/dataset.py

import pathlib
import os

def get_subfolder_dataset(data_dir,modality_conf):
    data_list = []
    for f in os.scandir(data_dir):

        if f.is_dir():
            subject_dict = {key:str(pathlib.Path(f.path).joinpath(f.name+modality_conf[key]['suffix'])) for key in modality_conf}
            data_list.append(subject_dict)
    return data_list

In [ ]:
%%bash

export BUNDLE_ROOT=nnUNetBundle
export PYTHONPATH=$PYTHONPATH:$BUNDLE_ROOT

python -m monai.bundle run \
    --config-file $BUNDLE_ROOT/configs/inference.yaml \
    --bundle-root $BUNDLE_ROOT \
    --data-dir $BUNDLE_ROOT/test_input \
    --output-dir $BUNDLE_ROOT/test_output \
    --logging-file $BUNDLE_ROOT/configs/logging.conf

## Utilities

### MONAI Bundle to nnUNet Conversion

To convert a MONAI Bundle to a nnUNet Bundle, we need to combine the MONAI checkpoint with the nnUNet checkpoint. This is done by loading the MONAI checkpoint and the nnUNet checkpoint, and updating the nnUNet model weights with the MONAI model weights.

In [ ]:
def subfiles(directory, prefix=None, suffix=None, join=True, sort=True):
    """
    List files in a directory with optional filtering by prefix and/or suffix.

    Parameters
    ----------
    directory : str
        The path to the directory to list files from.
    prefix : str, optional
        If specified, only files starting with this prefix will be included.
    suffix : str, optional
        If specified, only files ending with this suffix will be included.
    join : bool, optional
        If True, the directory path will be joined with the filenames. Default is True.
    sort : bool, optional
        If True, the list of files will be sorted. Default is True.

    Returns
    -------
    list of str
        A list of filenames (with full paths if `join` is True) that match the specified criteria.
    """

    files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    if prefix is not None:
        files = [f for f in files if f.startswith(prefix)]
    if suffix is not None:
        files = [f for f in files if f.endswith(suffix)]
    if join:
        files = [os.path.join(directory, f) for f in files]
    if sort:
        files.sort()
    return files


def convert_monai_bundle_to_nnunet(nnunet_config, bundle_path):

    nnunet_trainer = "nnUNetTrainer"
    nnunet_plans = "nnUNetPlans"

    if "nnunet_trainer" in nnunet_config:
        nnunet_trainer = nnunet_config["nnunet_trainer"]

    if "nnunet_plans" in nnunet_config:
        nnunet_plans = nnunet_config["nnunet_plans"]

    nnunet_model_folder = Path(os.environ["nnUNet_results"]).joinpath(
        maybe_convert_to_dataset_name(nnunet_config["dataset_name_or_id"]),
        f"{nnunet_trainer}__{nnunet_plans}__3d_fullres",
    )

    nnunet_preprocess_model_folder = Path(os.environ["nnUNet_preprocessed"]).joinpath(
        maybe_convert_to_dataset_name(nnunet_config["dataset_name_or_id"])
    )

    Path(nnunet_model_folder).joinpath("fold_0").mkdir(parents=True, exist_ok=True)

    nnunet_checkpoint = torch.load(f"{bundle_path}/models/nnunet_checkpoint.pth")
    latest_checkpoints = subfiles(
        Path(bundle_path).joinpath("models"), prefix="checkpoint_epoch", sort=True, join=False
    )
    epochs = []
    for latest_checkpoint in latest_checkpoints:
        epochs.append(int(latest_checkpoint[len("checkpoint_epoch=") : -len(".pt")]))

    epochs.sort()
    final_epoch = epochs[-1]
    monai_last_checkpoint = torch.load(f"{bundle_path}/models/checkpoint_epoch={final_epoch}.pt")

    best_checkpoints = subfiles(
        Path(bundle_path).joinpath("models"), prefix="checkpoint_key_metric", sort=True, join=False
    )
    key_metrics = []
    for best_checkpoint in best_checkpoints:
        key_metrics.append(str(best_checkpoint[len("checkpoint_key_metric=") : -len(".pt")]))

    key_metrics.sort()
    best_key_metric = key_metrics[-1]
    monai_best_checkpoint = torch.load(f"{bundle_path}/models/checkpoint_key_metric={best_key_metric}.pt")

    nnunet_checkpoint["optimizer_state"] = monai_last_checkpoint["optimizer_state"]

    nnunet_checkpoint["network_weights"] = odict()

    for key in monai_last_checkpoint["network_weights"]:
        nnunet_checkpoint["network_weights"][key] = monai_last_checkpoint["network_weights"][key]

    nnunet_checkpoint["current_epoch"] = final_epoch
    nnunet_checkpoint["logging"] = nnUNetLogger().get_checkpoint()
    nnunet_checkpoint["_best_ema"] = 0
    nnunet_checkpoint["grad_scaler_state"] = None

    torch.save(nnunet_checkpoint, Path(nnunet_model_folder).joinpath("fold_0", "checkpoint_final.pth"))

    nnunet_checkpoint["network_weights"] = odict()

    nnunet_checkpoint["optimizer_state"] = monai_best_checkpoint["optimizer_state"]

    for key in monai_best_checkpoint["network_weights"]:
        nnunet_checkpoint["network_weights"][key] = monai_best_checkpoint["network_weights"][key]

    torch.save(nnunet_checkpoint, Path(nnunet_model_folder).joinpath("fold_0", "checkpoint_best.pth"))

    shutil.move(f"{bundle_path}/models/dataset.json", nnunet_model_folder)
    shutil.move(f"{bundle_path}/models/plans.json", nnunet_model_folder)
    shutil.move(f"{nnunet_preprocess_model_folder}/dataset_fingerprint.json", nnunet_model_folder)
    shutil.move(f"{bundle_path}/models/nnunet_checkpoint.pth", nnunet_model_folder)
    shutil.move(f"{bundle_path}/models/checkpoint_epoch={final_epoch}.pt", f"{bundle_path}/models/model.pt")
    shutil.move(
        f"{bundle_path}/models/checkpoint_key_metric={best_key_metric}.pt", f"{bundle_path}/models/best_model.pt"
    )

In [ ]:
os.environ["nnUNet_results"] = "MONAI/Data/nnUNet/nnUNet_trained_models"
os.environ["nnUNet_raw"] = "MONAI/Data/nnUNet/nnUNet_raw_data_base"
os.environ["nnUNet_preprocessed"] = "MONAI/Data/nnUNet/nnUNet_preprocessed"

nnunet_config = {
    "dataset_name_or_id": "009",
    "nnunet_trainer": "nnUNetTrainer",
}

convert_monai_bundle_to_nnunet(nnunet_config, "nnUNetBundle")

### Testing the nnUNet Model

We now test the nnUNet model by performing inference on the test dataset and evaluating the model predictions.

In [ ]:
root_dir = "MONAI/Data"
nnunet_root_dir = os.path.join(root_dir, "nnUNet")

os.makedirs(nnunet_root_dir, exist_ok=True)

data_src_cfg = os.path.join(nnunet_root_dir, "data_src_cfg.yaml")
data_src = {
    "modality": "CT",
    "dataset_name_or_id": "09",
    "datalist": os.path.join(root_dir, "Task09_Spleen/msd_task09_spleen_folds.json"),
    "dataroot": os.path.join(root_dir, "Task09_Spleen"),
}

ConfigParser.export_config_file(data_src, data_src_cfg)

runner = nnUNetV2Runner(input_config=data_src_cfg, trainer_class_name="nnUNetTrainer", work_dir=nnunet_root_dir)

In [ ]:
runner.train_single_model(config="3d_fullres", fold=0, val="")

In [ ]:
runner.find_best_configuration(configs=["3d_fullres"], folds=[0], allow_ensembling=False, num_processes=1)

In [ ]:
runner.predict_ensemble_postprocessing(folds=[0], run_ensemble=False, run_postprocessing=False)

### nnUNet to MONAI Bundle Conversion

To convert a nnUNet trained Model to a MONAI Bundle, we need to separate the MONAI checkpoint from the nnUNet checkpoint. This is done by loading the nnUNet checkpoint and the MONAI checkpoint, and updating the MONAI model weights with the nnUNet model weights.

In [ ]:
os.environ["nnUNet_results"] = "MONAI/Data/nnUNet/nnUNet_trained_models"
os.environ["nnUNet_raw"] = "MONAI/Data/nnUNet/nnUNet_raw_data_base"
os.environ["nnUNet_preprocessed"] = "MONAI/Data/nnUNet/nnUNet_preprocessed"

nnunet_config = {
    "dataset_name_or_id": "009",
    "nnunet_trainer": "nnUNetTrainer_10epochs",
}

bundle_root = "nnUNetBundle"

convert_nnunet_to_monai_bundle(nnunet_config, bundle_root, 0)